In [ ]:
import requests
# !pip install google-search-results
# !pip install google-serp-api

from serpapi import GoogleSearch

def download_image(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
            print(f"Downloaded {filename}")
        return True
    else:
        print(f"Failed to download {filename}")
        return False

def download_images_from_sentences(sentences):
    api_key = "aeb54622bbb059e17c4065cae319d0fa4c50dc36e73b8257acf4541b41836c12"

    for i, sentence in enumerate(sentences):
        params = {
            "engine": "google_images",
            "q": sentence,
            "api_key": api_key,
        }

        search = GoogleSearch(params)
        results = search.get_dict()

        if "images_results" in results:
            images_results = results["images_results"]
            for j, image_result in enumerate(images_results):
                image_url = image_result["original"]
                filename = f"image_{i+1}.jpg"
                if download_image(image_url, filename):
                    print(f"Downloaded {filename}")
                    break  # Break out of the inner loop if download succeeds
            else:
                print(f"No image found for sentence {i+1}")
        else:
            print(f"No image results found for sentence {i+1}")
        
        print()  # Add a new line for visual separation

        

# Example usage
sentences = [
    'Cricket is believed to have originated in the 16th century in England and is now played in over 100 countries worldwide.',
    'The longest recorded cricket match lasted for 10 days. ',
    'It took place in 1939 between England and South Africa and ended in a draw.',
    'The highest individual score in international cricket is 400 runs, scored by Brian Lara from the West Indies against England in 2004.',
    'The fastest century in international cricket was scored by AB de Villiers from South Africa, who reached the milestone in just 31 balls in 2015.',
    'The Ashes is a famous cricket series played between England and Australia. It originated in 1882 and is one of the oldest rivalries in sports.',
]

download_images_from_sentences(sentences)


In [154]:

import moviepy.editor as mp
from gtts import gTTS
import os

def generate_voiceover(sentence, filename):
    tts = gTTS(text=sentence, lang='en')
    tts.save(filename)

def generate_subtitle(sentence, start_time, end_time, filename,i):
    start_time_str = format_time(start_time)
    end_time_str = format_time(end_time)
    subtitle = f"{i}\n{start_time_str} --> {end_time_str}\n{sentence}\n\n"
    with open(filename, 'a') as f:
        f.write(subtitle)

def format_time(seconds):
    hours = int(seconds / 3600)
    minutes = int((seconds % 3600) / 60)
    seconds = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

def create_video(sentences, image_files, video_filename):
    video_clips = []
    audio_clips = []
    subtitle_filename = 'subtitle.srt'
    FPS = 24  # Set the desired frames per second for the video
    subtitle_start_time = 0

    # Generate voiceovers and subtitles for each sentence
    for i, sentence in enumerate(sentences):
        image_file = image_files[i]
        voiceover_filename = f"voiceover_{i+1}.mp3"

        # Generate voiceover
        generate_voiceover(sentence, voiceover_filename)

        # Load voiceover and get its duration
        audio_clip = mp.AudioFileClip(voiceover_filename)
        audio_duration = audio_clip.duration

        # Generate subtitle
        generate_subtitle(sentence, subtitle_start_time, subtitle_start_time + audio_duration, subtitle_filename,i+1)

        # Load image and set duration based on voiceover duration
        image_clip = mp.ImageClip(image_file).set_duration(audio_duration)
        video_clips.append(image_clip)

        # Append voiceover clip to audio_clips list
        audio_clips.append(audio_clip)

        # Update the subtitle start time for the next iteration
        subtitle_start_time += audio_duration

os.remove('subtitle.srt')

# Example usage
sentences = [
    'Cricket is believed to have originated in the 16th century in England and is now played in over 100 countries worldwide.',
    'The longest recorded cricket match lasted for 10 days.',
    'It took place in 1939 between England and South Africa and ended in a draw.',
    'The highest individual score in international cricket is 400 runs, scored by Brian Lara from the West Indies against England in 2004.',
    'The fastest century in international cricket was scored by AB de Villiers from South Africa, who reached the milestone in just 31 balls in 2015.',
    'The Ashes is a famous cricket series played between England and Australia. It originated in 1882 and is one of the oldest rivalries in sports.'
]

image_files = ['image_1.jpg', 'image_2.jpg', 'image_3.jpg', 'image_4.jpg', 'image_5.jpg', 'image_6.jpg']

video_filename = 'cricket_video.mp4'

create_video(sentences, image_files, video_filename)



In [ ]:
import subprocess
from mutagen.mp3 import MP3

import moviepy.editor as mp
from gtts import gTTS
import os



# Define the path and filenames for the images and voiceovers
image_path = "./"
voiceover_path = "./"
image_files = ["image_1.jpg", "image_2.jpg", "image_3.jpg", "image_4.jpg", "image_5.jpg", "image_6.jpg"]
voiceover_files = ["voiceover_1.mp3", "voiceover_2.mp3", "voiceover_3.mp3", "voiceover_4.mp3", "voiceover_5.mp3", "voiceover_6.mp3"]

# Define the output video filename
output_video = "output.mp4"
def get_duration(voiceover_file):
    audio = MP3(voiceover_file)
    duration = audio.info.length
    return duration
# Create a list of video clips
video_clips = []
for image_file, voiceover_file in zip(image_files, voiceover_files):
    image_filepath = image_path + image_file
    voiceover_filepath = voiceover_path + voiceover_file

    # # Get the duration of the voiceover
    # cmd_duration = [
    #     "ffprobe",
    #     "-i", voiceover_filepath,
    #     "-show_entries", "format=duration",
    #     "-v", "quiet",
    #     "-of", "csv=p=0"
    # ]
    # result = subprocess.run(cmd_duration, capture_output=True, text=True)
    voiceover_duration = get_duration(voiceover_filepath)

    # Generate a video clip for each image
    cmd = [
        "ffmpeg",
        "-loop", "1",
        "-i", image_filepath,
        "-i", voiceover_filepath,
        "-c:v", "libx264",
        "-t", str(voiceover_duration),
        "-pix_fmt", "yuv420p",
        "-vf", "scale=1920:1080",
        "-shortest",
        "-y",
        "-strict", "-2",
        "-crf", "18",
        image_file.replace(".jpg", ".mp4")
    ]
    subprocess.run(cmd)

    # Add the video clip to the list
    video_clips.append(image_file.replace(".jpg", ".mp4"))

# Concatenate the video clips into a single video
cmd = [
    "ffmpeg",
    "-y",
    "-f", "concat",
    "-safe", "0",
    "-i", "ffmpeg_input.txt",
    "-c", "copy",
    output_video
]
with open("ffmpeg_input.txt", "w") as f:
    for video_clip in video_clips:
        f.write(f"file '{video_clip}'\n")
subprocess.run(cmd)

# Clean up temporary video clips
for video_clip in video_clips:
    subprocess.run(["rm", video_clip])

print("Video creation completed!")
